In [ ]:
import zeep
import pandas as pd
def getData(par_dict = {"KSRQ": "2018-02-26", "JSRQ": "2018-02-26", "YYB": "1", "TJFL": "0"},
            lookout_str='cxSqlKHCJTJ_GPXZ'):
    wsdl = 'http://10.21.2.75:8080/service/LBEBusiness?wsdl'
    client = zeep.Client(wsdl=wsdl)
    # ns0:loginResult(message: xsd:string, result: xsd:int, sessionId: xsd:string)
    sessionId = client.service.login("ZXQH_GPXZ", "GPXZ123321", "myapp", "plain", "")
    print(client.service.login("ZXQH_GPXZ", "GPXZ123321", "myapp", "plain", ""))
    factory = client.type_factory('ns0')
    string_type = client.get_type('xsd:string')
    lbParameter_type = client.get_type('ns0:lbParameter')
    queryOption_type = client.get_type('ns0:queryOption')
    '''		tmap.put("KSRQ", "2018-02-26");
    		tmap.put("JSRQ", "2018-02-26");
    		tmap.put("YYB", "1");
    		tmap.put("TJFL", "0");'''
    # par_dict = {"KSRQ": "2018-02-26", "JSRQ": "2018-02-26", "YYB": "1", "TJFL": "0"}
    params = []
    for key in par_dict:
        val = par_dict[key]
        temp_lbParameter = lbParameter_type(name=key, value=val)
        params.append(temp_lbParameter)

    valueOption_type = client.get_type('ns0:valueOption')
    valueOption = valueOption_type('VALUE')
    batchNo = 1
    batchSize = 100
    ans_records = []
    mqueryOption = queryOption_type(batchNo=batchNo, batchSize=batchSize, queryCount=True, valueOption=valueOption)

    ans = client.service.query(sessionId.sessionId, lookout_str, params, "", mqueryOption)
    ans_records.extend(ans.records)
    temp_meta = ans.metaData.colInfo
    print(temp_meta)
    ans_meta = [it['label'] for it in temp_meta]
    print(ans_meta)
    print(ans)
    while ((ans.result > 0) & ans.hasMore):
        batchNo += 1
        print('batchNo:{batchNo}'.format(batchNo=batchNo))

        mqueryOption = queryOption_type(batchNo=batchNo, batchSize=batchSize, queryCount=True, valueOption=valueOption)
        ans = client.service.query(sessionId.sessionId, lookout_str, params, "", mqueryOption)

        ans_records.extend(ans.records)
        # print(ans)
        print('batchNo:{batchNo}'.format(batchNo=batchNo))

    print(client.service.logout(sessionId.sessionId))
    return ans_records, ans_meta

In [2]:
par_dict = {"KSRQ": "2018-02-23", "JSRQ": "2018-02-23", 'TJLX': '2', 'ZB': '0', 'JYS': '1',
            'PM': '20'}
# 一．客户成交统计排名（CXSqlKHCJTJPM_GPXZ）
lookout_str = 'CXSqlKHCJTJPM_GPXZ'

records, meta = getData(par_dict, lookout_str)
clean_records = []
for a_record in records:
    clean_records.append(a_record.values)
# df = pd.DataFrame(data)
print(clean_records)
tab_cols = meta
# tab_cols = ['客户号', '客户姓名', '成交量', '成交金额', '排名', ]
df = pd.DataFrame(clean_records, columns=tab_cols)
repl = lambda m: m[-1:]
for x in df['客户姓名']:
    print("**" + x[-1:])
df['客户姓名'] = ["**" + x[-1:] for x in df['客户姓名']]

print(df['客户姓名'])
df['客户姓名'].str.replace(r'[W]+', repl=repl)
print(df['客户姓名'])
   

{
    'message': '成功',
    'result': 1,
    'sessionId': '77012d1adcd8b79b53455206bfd96200'
}


[{
    'label': '客户号',
    'length': 16,
    'name': '_CF_1',
    'scale': 0,
    'type': 0
}, {
    'label': '客户姓名',
    'length': 100,
    'name': '_CF_2',
    'scale': 0,
    'type': 0
}, {
    'label': '交易所',
    'length': 4,
    'name': '_CF_3',
    'scale': 0,
    'type': 0
}, {
    'label': '成交量',
    'length': 18,
    'name': '_CF_4',
    'scale': 2,
    'type': 1
}, {
    'label': '成交金额',
    'length': 18,
    'name': '_CF_5',
    'scale': 2,
    'type': 1
}, {
    'label': '排名',
    'length': 18,
    'name': '_CF_6',
    'scale': 2,
    'type': 1
}]
['客户号', '客户姓名', '交易所', '成交量', '成交金额', '排名']
{
    'message': '成功',
    'result': 1,
    'count': 20,
    'hasMore': False,
    'metaData': {
        'colInfo': [
            {
                'label': '客户号',
                'length': 16,
                'name': '_CF_1',
                'scale': 0,
                'type': 0
            },
            {
                'label': '客户姓名',
                'length': 100,
                

In [12]:

import pandas as pd
writer = pd.ExcelWriter('期货统计排名_交易所.xlsx', engine='xlsxwriter')
# tab_cols = ['排名', '客户姓名','成交量','成交金额',]
# df['客户姓名'] = ["**" + x[-1:] for x in df['客户姓名']]
print(df.columns)
# df = df.drop(['客户号',], axis=1)


v = df.values.tolist()
print(v)
# row = 1
# col = 0

# Iterate over the data and write it out row by row.
# for item, cost in (expenses):
#     worksheet.write(row, col,     item)
#     worksheet.write(row, col + 1, cost, money)
#     row += 1
     
     
workbook  = writer.book


# Add a header format.
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': False,
   
    
    'border': 1})
header_format.set_align('center')
header_format.set_align('vcenter')
header_format.set_border(1)
header_format.set_font_size(12)

cell_format = workbook.add_format({
    'bold': False,
    'text_wrap': False,
    
   
    'border': 1})

# cell_format.set_align()
cell_format.set_align('center')
cell_format.set_align('vcenter')
cell_format.set_border(1)

# Write the column headers with the defined format.
# for col_num, value in enumerate(df.columns.values):
#     worksheet.write(1, col_num, value, header_format)

# workbook.add_sheet("won't work")
worksheet2 = workbook.add_worksheet("期货排名2")
# worksheet2 = writer.sheets['期货排名2']
for index, row in df.iterrows():   # 获取每行的index、row
    for col_index, cell in enumerate(row):
        worksheet2.write(index+2, col_index, cell, cell_format)
for col_num, value in enumerate(df.columns.values):
    worksheet2.write(1, col_num, value, header_format)  
worksheet2.merge_range('A1:E1', '期货统计排名， 按所有品种交易量统计排名', header_format)
# Code to make the header red
colour_format = workbook.add_format()
colour_format.set_bg_color('#640000')
colour_format.set_font_color('white')

# Code to make the body blue    
table_body_format = workbook.add_format()
table_body_format.set_bg_color('blue')

# worksheet.set_row(0, 15, colour_format)
worksheet2.set_column('A:Z', 15, ) 
worksheet2.set_row(0, 30)
# font_fmt = workbook.add_format({'font_name': 'Arial', 'font_size': 10})
# header_fmt = workbook.add_format({'font_name': 'Arial', 'font_size': 10, 'bold': True})

# worksheet.set_column('A:A', None, font_fmt)
# worksheet.set_row(0, None, header_fmt)


# worksheet.merge_range('A1:D1', '期货统计排名， 按所有品种交易量统计排名', header_format)
writer.save()

Index(['客户姓名', '交易所', '成交量', '成交金额', '排名'], dtype='object')
[['**正', '大商所', '39680', '2166858285', '1'], ['**华', '大商所', '39312', '1905638250', '2'], ['**晓', '大商所', '11643', '580362985', '3'], ['**号', '大商所', '11310', '463305760', '4'], ['**本', '大商所', '6458', '27012585', '5'], ['**金', '大商所', '6240', '255636980', '6'], ['**敬', '大商所', '4116', '191030790', '7'], ['**彬', '大商所', '4080', '134455225', '8'], ['**芳', '大商所', '3800', '149221410', '9'], ['**金', '大商所', '3668', '208752380', '10'], ['**平', '大商所', '3530', '190421450', '11'], ['**红', '大商所', '2792', '151116700', '12'], ['**期', '大商所', '2760', '156170530', '13'], ['**本', '大商所', '2492', '91539700', '14'], ['**姗', '大商所', '2260', '177669690', '15'], ['**其', '大商所', '2200', '103152500', '16'], ['**玲', '大商所', '2160', '114988250', '17'], ['**舰', '大商所', '2000', '59117000', '18'], ['**宁', '大商所', '2000', '108150000', '18'], ['**际', '大商所', '1800', '52825980', '20']]


In [13]:
# XLSXtoPDF.py

# Program to convert the data from an XLSX file to PDF.
# Uses the openpyxl library and xtopdf.

# Author: Vasudev Ram - http://jugad2.blogspot.com
# Copyright 2015 Vasudev Ram.

# from openpyxl import load_workbook
# from PDFWriter import PDFWriter
# 
# workbook = load_workbook('fruits2.xlsx', guess_types=True, data_only=True)
# worksheet = workbook.active
# 
# pw = PDFWriter('fruits2.pdf')
# pw.setFont('Courier', 12)
# pw.setHeader('XLSXtoPDF.py - convert XLSX data to PDF')
# pw.setFooter('Generated using openpyxl and xtopdf')
# 
# ws_range = worksheet.iter_rows('A1:H13')
# for row in ws_range:
#     s = ''
#     for cell in row:
#         if cell.value is None:
#             s += ' ' * 11
#         else:
#             s += str(cell.value).rjust(10) + ' '
#     pw.writeLine(s)
# pw.savePage()
# pw.close()

In [15]:
# from win32com import client
# xlApp = client.Dispatch("Excel.Application")
# books = xlApp.Workbooks.Open('C:\\excel\\trial.xls')
# ws = books.Worksheets[0]
# ws.Visible = 1
# ws.ExportAsFixedFormat(0, 'C:\\excel\\trial.pdf')
import pandas as pd
import numpy as np
from fpdf import FPDF

# Creating a dataframe and saving as test.xlsx in current directory
df_1 = pd.DataFrame(np.random.randn(10, 2), columns=list('AB'))
writer = pd.ExcelWriter('test.xlsx')
df_1.to_excel(writer)
writer.save()

#read in the .xlsx file just created
df_2 = pd.read_excel('test.xlsx')

#creating a pdf in called test.pdf in the current directory
pdf = FPDF()
pdf.add_page()
pdf.set_xy(0, 0)
pdf.set_font('arial', 'B', 14)
pdf.cell(60)
pdf.cell(70, 10, 'Writing a PDF from python', 0, 2, 'C')
pdf.cell(-40)
pdf.cell(50, 10, 'Index Column', 1, 0, 'C')
pdf.cell(40, 10, 'Col A', 1, 0, 'C')
pdf.cell(40, 10, 'Col B', 1, 2, 'C')
pdf.cell(-90)
pdf.set_font('arial', '', 12)
for i in range(0, len(df_2)-1):
    col_ind = str(i)
    col_a = str(df_2.A.ix[i])
    col_b = str(df_2.B.ix[i])
    pdf.cell(50, 10, '%s' % (col_ind), 1, 0, 'C')
    pdf.cell(40, 10, '%s' % (col_a), 0, 0, 'C')
    pdf.cell(40, 10, '%s' % (col_b), 0, 2, 'C')
    pdf.cell(-90)
pdf.output('test.pdf', 'F')


C:\Users\niejn\PycharmProjects\scrapy_demo\venv\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\niejn\PycharmProjects\scrapy_demo\venv\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


''

In [26]:
import os
import comtypes.client

SOURCE_DIR = 'C:/Users/IEUser/Documents/jscript/test/resources/root3'
TARGET_DIR = 'C:/Users/IEUser/Documents/jscript'

app = comtypes.client.CreateObject('Excel.Application')
app.Visible = False

infile = os.path.join(os.path.abspath(SOURCE_DIR), 'spreadsheet1.xlsx')
outfile = os.path.join(os.path.abspath(TARGET_DIR), 'spreadsheet1.pdf')

infile = 'C:/Users/niejn/PycharmProjects/scrapy_demo/期货统计排名.xlsx'
outfile = 'C:/Users/niejn/PycharmProjects/scrapy_demo/test.pdf'

doc = app.Workbooks.Open(infile)
doc.ExportAsFixedFormat(0, outfile, 1, 0)
doc.Close()

app.Quit()

COMError: (-2147018887, '指定的文件是只读文件。', (None, None, None, 0, None))